In [4]:
import random
from functools import reduce

# Part I: MPC Sum using Secret Sharing
# Each participant has a private value (e.g., salary)
private_values = {
    "Ahmed": 100,
    "Ali": 70,
    "Rashid": 50
}

# Function to split a secret number into random shares
def split_into_shares(secret, n=3):
    shares = [random.randint(1, 50) for _ in range(n-1)]
    final_share = secret - sum(shares)
    shares.append(final_share)
    return shares

# Each participant splits their secret into 3 shares
all_shares = {}
names = list(private_values.keys())

for name in names:
    all_shares[name] = split_into_shares(private_values[name], 3)

# Distribute shares among participants
# Each one receives one share from each participant
received = {name: [] for name in names}

for sender in names:
    for i, receiver in enumerate(names):
        received[receiver].append(all_shares[sender][i])

# Each participant sums received shares
local_sums = {name: sum(received[name]) for name in names}

# Global sum = sum of local sums
global_sum = sum(local_sums.values())

print("\n--- Part I: MPC Sum Result ---")
print("Local sums:", local_sums)
print("Final total (should be 220):", global_sum)
# Note: global_sum equals the real total because shares cancel correctly.


# Part II: Shamir Secret Sharing (k, n) Implementation
# Polynomial evaluation function
def evaluate_polynomial(coeffs, x):
    return sum([coeff * (x ** i) for i, coeff in enumerate(coeffs)])

#Generate shares using Shamir’s Secret Sharing (k, n)
#secret = the value that needs to be protected
#k = the minimum number of shares required to reconstruct the secret (threshold)
#n = the total number of shares to be generated
def shamir_split(secret, k, n):
    coeffs = [secret] + [random.randint(1, 20) for _ in range(k - 1)]
    shares = []
    for x in range(1, n + 1):
        y = evaluate_polynomial(coeffs, x)
        shares.append((x, y))
    return shares

# Reconstruct the secret using any k shares
def shamir_reconstruct(shares):
    def lagrange(i):
        xi, yi = shares[i]
        total = 1
        for j in range(len(shares)):
            if i != j:
                xj, _ = shares[j]
                total *= xj / (xj - xi)
        return yi * total

    return round(sum(lagrange(i) for i in range(len(shares))))

secret = 123
k = 3
n = 5

shares = shamir_split(secret, k, n)
print("\n--- Part II: Shamir Secret Sharing ---")
print("Generated Shares:", shares)

# Use any 3 shares to rebuild the secret
chosen = shares[:3]
recovered = shamir_reconstruct(chosen)

print("Recovered Secret:", recovered)



# Part III: Garbled Circuit (AND Gate)

# Simple example of garbled circuit for AND gate
# Encryption labels (fake random labels for demonstration)
labels = {
    "A0": "X15", "A1": "X92",
    "B0": "Y44", "B1": "Y78",
    "OUT0": "Z01", "OUT1": "Z99"
}

# Garbled table for AND gate
garbled_table = {
    (labels["A0"], labels["B0"]): labels["OUT0"],
    (labels["A0"], labels["B1"]): labels["OUT0"],
    (labels["A1"], labels["B0"]): labels["OUT0"],
    (labels["A1"], labels["B1"]): labels["OUT1"],
}

# Input bits
A = 1   # Omar's bit
B = 0   # Nancy's bit

# Get encrypted labels
A_label = labels[f"A{A}"]
B_label = labels[f"B{B}"]

# Compute encrypted output
encrypted_output = garbled_table[(A_label, B_label)]

print("\n--- Part III: Garbled Circuit AND Gate ---")
print("Encrypted Output Label:", encrypted_output)
print("Real AND result:", A & B)


--- Part I: MPC Sum Result ---
Local sums: {'Ahmed': 79, 'Ali': 126, 'Rashid': 15}
Final total (should be 220): 220

--- Part II: Shamir Secret Sharing ---
Generated Shares: [(1, 149), (2, 205), (3, 291), (4, 407), (5, 553)]
Recovered Secret: 123

--- Part III: Garbled Circuit AND Gate ---
Encrypted Output Label: Z01
Real AND result: 0
